In [10]:
import sqlite3
conn = sqlite3.connect('challenge.db', timeout=5.0)
cursor = conn.cursor()

In [11]:
# On first glance, it seemed as if there were a lot more variables in the variables_list file than were accounted for in the table.

In [12]:
cursor.execute("SELECT COUNT(*) FROM variable_list")
row_count = cursor.fetchone()[0]

cursor.execute("SELECT \"Variable Code\" FROM variable_list")
variable_codes = [row[0] for row in cursor.fetchall()]

In [13]:
cursor.execute("PRAGMA table_info(five_hundred_cities)")
columns_500c = cursor.fetchall()
cursor.execute("PRAGMA table_info(access)")
columns_access = cursor.fetchall()

total_column_count = len(columns_500c) + len(columns_access)

In [14]:
print(f"Variables represented by rows in variables_list: {row_count}")
print(f"Variables represented in the columns: {total_column_count}")

Variables represented by rows in variables_list: 281
Variables represented in the columns: 109


In [7]:
# This confirmed that there were a number of variables for which I was missing values.

# Some of these missing variables seemed relevant to the assignment, like the number of grocery stores per county,
# so I downloaded the complete dataset from the USDA.

# Specifically, I wanted the sheets: "STORES", "RESTAURANTS", "ASSISTANCE", "INSECURITY", "PRICES_TAXES", "LOCAL", "HEALTH", and "SOCIOECONOMIC".

In [23]:
import pandas as pd
import os

csv_dir = 'usda_csv'

csv_files = [
    'stores.csv',
    'restaurants.csv',
    'assistance.csv',
    'insecurity.csv',
    'prices_taxes.csv',
    'local.csv',
    'health.csv',
    'socioeconomic.csv'
]


cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

if len(tables) == 3:
    for file in csv_files:
        file_path = os.path.join(csv_dir, file)
        table_name = os.path.splitext(file)[0]
    
        df = pd.read_csv(file_path)
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        conn.commit()
else:
    print("The missing tables have been added to the database.")
    
conn.close()

The missing tables have already been added to the database.
